In [32]:
from google.cloud import bigquery
import urllib.request
import urllib.parse
import json
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()

apikey = os.getenv('FMP_API_KEY')
project_id = os.getenv('GCP_PROJECT_ID')

In [33]:
client = bigquery.Client(project=project_id)
db = 'stock_data'
table = 'raw_stock_data'
table_id = f"{project_id}.{db}.{table}"
table_id

'stock-prices-442513.stock_data.raw_stock_data'

In [55]:
symbol = 'AAPL'

query_string = f"""
    SELECT `date`
    FROM `{table_id}`
    WHERE `symbol` = @symbol
    ORDER BY `date` DESC
    LIMIT 1
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter('symbol', 'STRING', symbol)
    ]
)
query_job = client.query(query_string, job_config=job_config)
results = query_job.result()

In [56]:
for row in results:
    print(row.date)

2024-11-22


In [34]:
def historical_url(ticker, apikey, from_date=None):
    url = 'https://financialmodelingprep.com/api/v3/historical-price-full/'

    query_params = {
        'apikey': apikey
    }

    if from_date is not None:
        query_params['from'] = from_date

    encoded_params = urllib.parse.urlencode(query_params)

    full_url = f"{url}{ticker}?{encoded_params}"

    return full_url

response = urllib.request.urlopen(historical_url('AAPL', apikey, '2024-11-20'))

data = json.loads(response.read())

In [35]:
for item in data['historical']:
    item['symbol'] = data['symbol']
    item['timestamp'] = datetime.now().isoformat()

rows_to_insert = data['historical']
errors = client.insert_rows_json(table_id, rows_to_insert)

if errors == []:
    print("Data inserted successfully.")
else:
    print("Encountered errors while inserting rows:", errors)

Data inserted successfully.
